In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [3]:
class FaceEmbeddingCNN(nn.Module):
    def __init__(self):
        super(FaceEmbeddingCNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),  # Conv1
            nn.ReLU(),
            nn.MaxPool2d(2),  # 96x96 → 48x48

            nn.Conv2d(16, 32, kernel_size=3, padding=1),  # Conv2
            nn.ReLU(),
            nn.MaxPool2d(2),  # 48x48 → 24x24

            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # Conv3
            nn.ReLU(),
            nn.MaxPool2d(2)   # 24x24 → 12x12
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 12 * 12, 128),
            nn.ReLU(),
            nn.Linear(128, 64)  # 64-dimensional embedding
        )

    def forward(self, x):
        x = self.cnn(x)
        x = self.fc(x)
        return x


In [4]:
transform = transforms.Compose([
    transforms.Grayscale(),          # Convertir a blanco y negro
    transforms.Resize((96, 96)),     # Redimensionar a 96x96
    transforms.ToTensor()            # Convertir a tensor PyTorch
])

In [5]:
import os

# Crear una instancia del modelo
model = FaceEmbeddingCNN()
model.eval()  # Cambia el modelo a modo evaluación

# Ruta de la carpeta con imágenes
folder_path = 'rostros'

# Iterar sobre las imágenes de la carpeta
for file_name in os.listdir(folder_path):
    if file_name.endswith(('.jpg', '.png', '.jpeg')):
        image_path = os.path.join(folder_path, file_name)
        image = Image.open(image_path)
        image_tensor = transform(image).unsqueeze(0)

        with torch.no_grad():
            embedding = model(image_tensor)

        print(f"📷 Imagen: {file_name}")
        print(embedding.numpy().flatten())  # Mostrar como vector 1D
        print("-" * 50)


📷 Imagen: AdrianCisneros.jpg
[ 0.04274729 -0.02317246  0.00833382 -0.03561298  0.06741799  0.08596631
  0.03655328 -0.08362328 -0.09837392  0.03290115  0.07501826 -0.01290081
 -0.00243112 -0.06995141 -0.03003571 -0.00515452  0.0529692  -0.00186767
 -0.05155282  0.03475223 -0.07475919  0.00669636 -0.00572243 -0.05907258
  0.04713828  0.04723639  0.07853578 -0.02556294 -0.017235   -0.08655655
  0.03644713  0.02662813 -0.02851487  0.06096233  0.03340367  0.05223657
  0.0577184   0.05728784 -0.01470131  0.04582331 -0.02179446  0.01071381
 -0.03338606 -0.02485287  0.09453254  0.08399533 -0.00313653 -0.08915667
  0.03100945 -0.05229191 -0.084519    0.04761726 -0.00607656 -0.00273522
 -0.05029227  0.00145286  0.0001476   0.04971299 -0.05948689 -0.02852853
  0.06858203 -0.07785995  0.07426398 -0.05559987]
--------------------------------------------------
📷 Imagen: AlejandroOliden.jpg
[ 0.04298389 -0.02287461  0.01379389 -0.03855     0.06412708  0.08698283
  0.03675673 -0.08188252 -0.09481484 

In [6]:
def comparar_embeddings(emb1, emb2, threshold=0.6):
    """
    Compara dos vectores de características (embeddings) usando distancia euclidiana.
    Retorna True si la distancia es menor al umbral (es decir, si hay coincidencia).
    """
    distancia = torch.norm(emb1 - emb2).item()
    print(f"Distancia entre embeddings: {distancia:.4f}")
    return distancia < threshold


In [12]:
# Cargar dos imágenes
img1 = Image.open("rostros/AdrianCisneros.jpg")
img2 = Image.open("rostros/JaimeLescano.jpg")  # Cámbialo por otra imagen que quieras probar

# Procesarlas
img1_tensor = transform(img1).unsqueeze(0)
img2_tensor = transform(img2).unsqueeze(0)

# Obtener embeddings
with torch.no_grad():
    emb1 = model(img1_tensor)
    emb2 = model(img2_tensor)

# Comparar
if comparar_embeddings(emb1, emb2):
    print("🔒 Coincidencia: Mismo rostro detectado.")
else:
    print("❌ No coinciden: Rostros diferentes.")


Distancia entre embeddings: 0.0232
🔒 Coincidencia: Mismo rostro detectado.


Sale "Mismo rostro detectado" cuando son rostros diferentes, iremos entrenando la CNN.

In [10]:
from torchvision import transforms
from PIL import Image
import os

augmentation = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(100, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor()
])

# Aumentar cada imagen 10 veces
output_folder = "dataset_augmented"
os.makedirs(output_folder, exist_ok=True)

for person_name in os.listdir("dataset"):
    person_path = os.path.join("dataset", person_name)
    if not os.path.isdir(person_path):
        continue

    output_person_path = os.path.join(output_folder, person_name)
    os.makedirs(output_person_path, exist_ok=True)

    for file in os.listdir(person_path):
        if file.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(person_path, file)
            image = Image.open(image_path)

            for i in range(10):
                augmented = augmentation(image)
                save_path = os.path.join(output_person_path, f"{file[:-4]}_aug_{i}.jpg")
                transforms.ToPILImage()(augmented).save(save_path)
